In [12]:

{
 "cells": [
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "# Bangalore Housing Price Prediction"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": None,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Importing the necessary libraries\n",
    "import numpy as np\n",
    "import pandas as pd\n",
    "import matplotlib.pyplot as plt\n",
    "import matplotlib\n",
    "import seaborn as sns\n",
    "from sklearn.preprocessing import OneHotEncoder\n",
    "from sklearn.pipeline import Pipeline\n",
    "from sklearn.preprocessing import StandardScaler\n",
    "from sklearn.base import BaseEstimator, TransformerMixin\n",
    "from sklearn.pipeline import FeatureUnion\n",
    "from sklearn.impute import SimpleImputer\n",
    "from sklearn.compose import ColumnTransformer\n",
    "from sklearn.model_selection import train_test_split, cross_val_score\n",
    "from sklearn.metrics import mean_squared_error\n",
    "from xgboost import XGBRegressor\n",
    "import pickle\n",
    "import json\n",
    "pd.options.display.max_columns = None\n",
    "pd.set_option('display.max_rows', 500)\n",
    "%matplotlib inline\n",
    "matplotlib.rcParams[\"figure.figsize\"] = (10,5)"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": None,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Load data\n",
    "df = pd.read_csv('Bengaluru_House_Data.csv')\n",
    "df.head()"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": None,
   "metadata": {},
   "outputs": [],
   "source": [
    "# =========================\n",
    "# Basic cleaning (your provided code)\n",
    "# =========================\n",
    "# drop columns we won't use and rows with missing values\n",
    "df = df.drop(['society','balcony'], axis=1)\n",
    "df = df.dropna()\n",
    "\n",
    "# size -> bhk (e.g., \"2 BHK\" -> 2)\n",
    "df['bhk'] = df['size'].apply(lambda x: int(x.split(' ')[0]))\n",
    "df = df.drop('size', axis=1)\n",
    "\n",
    "# convert total_sqft (handle \"low-high\" ranges)\n",
    "def convert_sqft_to_num(x):\n",
    "    try:\n",
    "        parts = str(x).split('-')\n",
    "        if len(parts) == 2:\n",
    "            return (float(parts[0]) + float(parts[1]))/2\n",
    "        return float(x)\n",
    "    except:\n",
    "        return None\n",
    "\n",
    "df['total_sqft'] = df['total_sqft'].apply(convert_sqft_to_num)\n",
    "df = df[df.total_sqft.notnull()]\n",
    "\n",
    "# keep only reasonable homes (simple rule used by the tutorial)\n",
    "df = df[~(df.total_sqft/df.bhk < 300)]\n",
    "\n",
    "# strip spaces in location names\n",
    "df['location'] = df['location'].apply(lambda x: x.strip())\n",
    "\n",
    "# group rare locations to \"other\"\n",
    "loc_counts = df['location'].value_counts()\n",
    "rare_locs = loc_counts[loc_counts <= 10].index\n",
    "df['location'] = df['location'].apply(lambda x: 'other' if x in rare_locs else x)\n",
    "\n",
    "# remove price-per-sqft outliers by location (std-dev rule)\n",
    "df['price_per_sqft'] = df['price']*100000/df['total_sqft']\n",
    "def remove_pps_outliers(d):\n",
    "    out = []\n",
    "    for loc, sub in d.groupby('location'):\n",
    "        m = sub.price_per_sqft.mean()\n",
    "        s = sub.price_per_sqft.std()\n",
    "        keep = sub[(sub.price_per_sqft > (m - s)) & (sub.price_per_sqft <= (m + s))]\n",
    "        out.append(keep)\n",
    "    return pd.concat(out, ignore_index=True)\n",
    "\n",
    "df = remove_pps_outliers(df)\n",
    "\n",
    "# remove bhk outliers\n",
    "def bhk_outlier_remover(df):\n",
    "    exclude_indices = np.array([])\n",
    "    for location, location_df in df.groupby('location'):\n",
    "        bhk_stats = {}\n",
    "        for bhk, bhk_df in location_df.groupby('bhk'):\n",
    "            bhk_stats[bhk] = {\n",
    "                'mean': np.mean(bhk_df.price_per_sqft),\n",
    "                'std': np.std(bhk_df.price_per_sqft),\n",
    "                'count': bhk_df.shape[0]\n",
    "            }\n",
    "        for bhk, bhk_df in location_df.groupby('bhk'):\n",
    "            stats = bhk_stats.get(bhk-1)\n",
    "            if stats and stats['count']>5:\n",
    "                exclude_indices = np.append(exclude_indices, \n",
    "                                            bhk_df[bhk_df.price_per_sqft < (stats['mean'])].index.values)\n",
    "    return df.drop(exclude_indices, axis='index')\n",
    "\n",
    "df = bhk_outlier_remover(df)\n",
    "\n",
    "# drop helper column\n",
    "df = df.drop(columns='price_per_sqft')\n",
    "\n",
    "# keep homes where baths aren't extreme\n",
    "df = df[df.bath < df.bhk + 2]\n",
    "\n",
    "# Binarize availability (map dates like '19-Dec' to 'Soon to be Vacated')\n",
    "df['availability'] = df['availability'].apply(lambda x: 'Ready To Move' if x == 'Ready To Move' else 'Soon to be Vacated')"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": None,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Prepare features and target\n",
    "X = df.drop('price', axis=1)\n",
    "y = df.price\n",
    "\n",
    "# One-hot encoding with drop_first=True\n",
    "dummies = pd.get_dummies(X[['location', 'area_type', 'availability']], drop_first=True)\n",
    "X = pd.concat([X[['total_sqft', 'bath', 'bhk']], dummies], axis=1)\n",
    "\n",
    "# Train-test split\n",
    "X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=10)\n",
    "\n",
    "# Train XGBoost model\n",
    "xgb_reg = XGBRegressor()\n",
    "xgb_reg.fit(X_train, y_train)\n",
    "print('Train score:', xgb_reg.score(X_train, y_train))\n",
    "print('Test score:', xgb_reg.score(X_test, y_test))\n",
    "\n",
    "# Cross-validation\n",
    "cv_scores = cross_val_score(xgb_reg, X, y, cv=5)\n",
    "print('CV mean score:', cv_scores.mean())"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": None,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Export model and columns\n",
    "with open('banglore_home_prices_model.pickle', 'wb') as f:\n",
    "    pickle.dump(xgb_reg, f)\n",
    "\n",
    "columns = {\n",
    "    'availability_columns': sorted(df['availability'].unique()),\n",
    "    'area_columns': sorted(df['area_type'].unique()),\n",
    "    'location_columns': sorted(df['location'].unique())\n",
    "}\n",
    "with open('columns.json', 'w') as f:\n",
    "    json.dump(columns, f)"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": None,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Test prediction\n",
    "def predict_price(location, area_type, availability, sqft, bhk, bath):\n",
    "    loc_index = np.where(X.columns==f'location_{location}')[0][0] if f'location_{location}' in X.columns else -1\n",
    "    area_index = np.where(X.columns==f'area_type_{area_type}')[0][0] if f'area_type_{area_type}' in X.columns else -1\n",
    "    avail_index = np.where(X.columns==f'availability_{availability}')[0][0] if f'availability_{availability}' in X.columns else -1\n",
    "\n",
    "    x = np.zeros(len(X.columns))\n",
    "    x[0] = sqft\n",
    "    x[1] = bath\n",
    "    x[2] = bhk\n",
    "    if loc_index >= 0:\n",
    "        x[loc_index] = 1\n",
    "    if area_index >= 0:\n",
    "        x[area_index] = 1\n",
    "    if avail_index >= 0:\n",
    "        x[avail_index] = 1\n",
    "\n",
    "    return xgb_reg.predict([x])[0]\n",
    "\n",
    "# Test with first row (expected ~39.07 lakhs)\n",
    "print(predict_price('Electronic City Phase II', 'Super built-up  Area', 'Soon to be Vacated', 1056, 2, 2))"
   ]
  }
 ],
 "metadata": {
  "kernelspec": {
   "display_name": "Python 3",
   "language": "python",
   "name": "python3"
  },
  "language_info": {
   "codemirror_mode": {
    "name": "ipython",
    "version": 3
   },
   "file_extension": ".py",
   "mimetype": "text/x-python",
   "name": "python",
   "nbconvert_exporter": "python",
   "pygments_lexer": "ipython3",
   "version": "3.7.7"
  }
 },
 "nbformat": 4,
 "nbformat_minor": 4
}

{'cells': [{'cell_type': 'markdown',
   'metadata': {},
   'source': ['# Bangalore Housing Price Prediction']},
  {'cell_type': 'code',
   'execution_count': None,
   'metadata': {},
   'outputs': [],
   'source': ['# Importing the necessary libraries\n',
    'import numpy as np\n',
    'import pandas as pd\n',
    'import matplotlib.pyplot as plt\n',
    'import matplotlib\n',
    'import seaborn as sns\n',
    'from sklearn.preprocessing import OneHotEncoder\n',
    'from sklearn.pipeline import Pipeline\n',
    'from sklearn.preprocessing import StandardScaler\n',
    'from sklearn.base import BaseEstimator, TransformerMixin\n',
    'from sklearn.pipeline import FeatureUnion\n',
    'from sklearn.impute import SimpleImputer\n',
    'from sklearn.compose import ColumnTransformer\n',
    'from sklearn.model_selection import train_test_split, cross_val_score\n',
    'from sklearn.metrics import mean_squared_error\n',
    'from xgboost import XGBRegressor\n',
    'import pickle\n',
    